In [1]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
import sys
from numpy.random import rand

from yaml import safe_load, load, dump
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper   

sys.path.insert(0, '/home/phyto/planktonSDM/functions/')

from predict import predict
from functions import example_data

In [2]:
with open('/home/phyto/planktonSDM/configuration/example_model_config.yml', 'r') as f:
    model_config = load(f, Loader=Loader)

X, y = example_data(y_name =  "Coccolithus pelagicus", n_samples=500, n_features=5, noise=20, random_state=model_config['seed'])

envdata = pd.DataFrame({"no3": rand(50), "mld": rand(50), "par": rand(50), "o2": rand(50), "temp": rand(50),
                        "lat": range(0,50, 1), "lon": 50*[1]})
envdata.set_index(['lat', 'lon'], inplace=True)


In [3]:
m = predict(X, y, envdata, model_config)
m.make_prediction()

1
/home/phyto/CoccoML/ModelOutput/rf/scoring/
/home/phyto/CoccoML/ModelOutput/rf/model/
predicting classifier


/home/phyto/anaconda3/envs/my-geopandas-env-2/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


finished
execution time: 0.39756250381469727 seconds
